In [3]:
import numpy as np
import cv2
import os
import matplotlib.pylab as plt
from glob import glob
import numpy.ma as ma


this_path = os.path.dirname('/home-local/poram.extra.nobkp/VOT/DAVIS/Annotations/480p/')
list_subfolders_with_paths = [f.path for f in os.scandir(this_path) if f.is_dir()]

In [5]:
for pathh in list_subfolders_with_paths:
    INPUT_PATH = pathh
    filenames = glob(os.path.join(INPUT_PATH, "*.png"))
    
    images=[]
    for f in filenames:
        img = cv2.imread(f)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        images.append(img)

    unique_items=np.unique(images[0])
    print(INPUT_PATH)
    print(unique_items)
    
    d=[]
    for m in unique_items:
        if m != 0:
            s=[]
            for im in images:
                masked_im=ma.masked_equal(im, m)
                if masked_im.mask.shape==():
                    masked_im=np.ma.masked_all_like(im)
                    masked_im.mask=~masked_im.mask
                s.append(masked_im.mask)
            #s.pop()
            d.append(s) #len[d]=number of unique items in the first image.each element of 
                        #d contains masks for all images for an item.    

    for j in range(len(d)):
        os.mkdir(INPUT_PATH+'/masks-'+str(j)+'/')
        for i,ss in enumerate(d[j]):
            plt.imsave(os.path.join(INPUT_PATH+'/masks-'+str(j)+'/'+ str(i)+'.png'),1*ss)
            
    for k in range(len(d)):
        for ii,sss in enumerate(d[k]):
            imgc = cv2.imread(os.path.join(INPUT_PATH+'/masks-'+str(k)+'/'+ str(ii)+'.png'),0)
            ret,thresh = cv2.threshold(imgc,127,255,0)
            contours,hierarchy = cv2.findContours(thresh, 1, 2)
            if len(contours)>0:
                cnt = contours[-1]#contours[0]
                   
                rect = cv2.minAreaRect(cnt)
                box = cv2.boxPoints(rect)
                box = np.int0(box)
            else:
                box=np.asarray([[0,0],[0,0],[0,0],[0,0]])
            
            plt.imsave(os.path.join(INPUT_PATH+'/masks-'+str(k)+'/m-'+ str(ii)+'.png'),cv2.drawContours(imgc,[box],0,(0,0,255),2))
            np.savetxt(os.path.join(INPUT_PATH+'/masks-'+str(k)+'/'+ str(ii)+'.txt'),box,fmt='%d')

/gel/usr/poram/Desktop/Rotated-BoundingBox/s/1
[  0  15  38  53  75 113]
/gel/usr/poram/Desktop/Rotated-BoundingBox/s/2
[  0  38  75 113]
